In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import wandb
import warnings
warnings.filterwarnings(action='ignore')


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
model_name = 'detectors_cascade_rcnn_r50_1x'
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/detectors_cascade_rcnn_r50_1x_coco.py')

root='../dataset/'
#print(cfg.data.train)
#print(cfg.data.val)
# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'split_train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'split_valid.json' # val json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 8

cfg.image_norm_config = dict(
                    mean=[127.49413776397705, 127.43779182434082, 127.46098327636719], 
                    std=[73.86627551077616, 73.88234865304638, 73.8944344154546], 
                    to_rgb=True
                    )

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/detectors_cascade_rcnn_r50_1x_coco'

cfg.model.roi_head.bbox_head[0]['num_classes'] = 10
cfg.model.roi_head.bbox_head[1]['num_classes'] = 10
cfg.model.roi_head.bbox_head[2]['num_classes'] = 10

cfg.log_config.hooks.append(dict(
                                type='WandbLoggerHook',
                                init_kwargs=dict(
                                            project='mmdetection',
                                            name=f'13-{model_name}',
                                            entity='passion-ate'
                                            )
                                )
                            )
#cfg.model.bbox_head['num_classes'] = 10
cfg.evaluation['save_best'] = 'bbox_mAP_50'
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=2, interval=1)

In [3]:
albu_train_transforms = [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.0625,
        scale_limit=0.0,
        rotate_limit=0,
        interpolation=1,
        p=0.5),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0)
        ],
        p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0)
        ],
        p=0.1),
]
cfg.train_pipeline[3] = dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='coco',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),

In [4]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [5]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3796, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3605  | 1 [Paper]     | 5775  | 2 [Paper pack]  | 815   | 3 [Metal]   | 850   | 4 [Glass]    | 892   |
| 5 [Plastic]       | 2674  | 6 [Styrofoam] | 1148  | 7 [Plastic bag] | 4707  | 8 [Battery] | 145   | 9 [Clothing] | 425   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [6]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2021-09-29 04:19:30,134 - mmdet - INFO - Use load_from_torchvision loader
2021-09-29 04:19:30,396 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset_l.weight, layer2.1.conv2.offset_l.bias, layer2.2.conv2.weight_diff, layer2.2.con

In [7]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2021-09-29 04:20:07,234 - mmdet - INFO - Start running, host: root@03f1f0147f93, work_dir: /opt/ml/detection/mmdetection/work_dirs/detectors_cascade_rcnn_r50_1x_coco
2021-09-29 04:20:07,235 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook              

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


wandb: Currently logged in as: passion-ate (use `wandb login --relogin` to force relogin)


2021-09-29 04:21:20,685 - mmdet - INFO - Epoch [1][50/475]	lr: 1.978e-03, eta: 2:07:22, time: 1.353, data_time: 0.057, memory: 10019, loss_rpn_cls: 0.4706, loss_rpn_bbox: 0.0605, s0.loss_cls: 0.6154, s0.acc: 87.0054, s0.loss_bbox: 0.1407, s1.loss_cls: 0.1966, s1.acc: 93.6479, s1.loss_bbox: 0.0426, s2.loss_cls: 0.1513, s2.acc: 82.9390, s2.loss_bbox: 0.0109, loss: 1.6887, grad_norm: 11.5233
2021-09-29 04:22:25,378 - mmdet - INFO - Epoch [1][100/475]	lr: 3.976e-03, eta: 2:03:30, time: 1.294, data_time: 0.010, memory: 10019, loss_rpn_cls: 0.1592, loss_rpn_bbox: 0.0417, s0.loss_cls: 0.3572, s0.acc: 92.4297, s0.loss_bbox: 0.1765, s1.loss_cls: 0.1100, s1.acc: 96.0815, s1.loss_bbox: 0.0679, s2.loss_cls: 0.0326, s2.acc: 98.0918, s2.loss_bbox: 0.0112, loss: 0.9562, grad_norm: 2.9297
2021-09-29 04:23:30,467 - mmdet - INFO - Epoch [1][150/475]	lr: 5.974e-03, eta: 2:01:44, time: 1.302, data_time: 0.010, memory: 10020, loss_rpn_cls: 0.1495, loss_rpn_bbox: 0.0498, s0.loss_cls: 0.3677, s0.acc: 91.3306

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1087/1087, 7.2 task/s, elapsed: 150s, ETA:     0s

...

2021-09-29 06:54:38,039 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.18s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.09s).
Accumulating evaluation results...
DONE (t=0.38s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.530
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.437
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.183
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.604
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.604
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.604
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxD

2021-09-29 06:54:42,228 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_50_epoch_12.pth.
2021-09-29 06:54:42,229 - mmdet - INFO - Best bbox_mAP_50 is 0.5300 at 12 epoch.
2021-09-29 06:54:42,243 - mmdet - INFO - Epoch(val) [12][1087]	bbox_mAP: 0.4110, bbox_mAP_50: 0.5300, bbox_mAP_75: 0.4370, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.1830, bbox_mAP_l: 0.4420, bbox_mAP_copypaste: 0.411 0.530 0.437 0.000 0.183 0.442


learning_rate,0.0002
momentum,0.9
train/grad_norm,1.89723
train/loss,0.40164
train/loss_rpn_bbox,0.01775
train/loss_rpn_cls,0.01477
train/s0.acc,95.99951
train/s0.loss_bbox,0.07222
train/s0.loss_cls,0.11484
train/s1.acc,96.69558
train/s1.loss_bbox,0.06597


learning_rate,▂▄▆████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,█▂▂▂▂▂▁▁▁▁▁▂▁▁▂▁▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▅▅▅▅▄▄▄▄▄▃▃▄▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▆▆▅▆▄▄▄▄▅▃▃▃▄▃▃▄▂▃▃▂▃▂▃▂▃▃▂▂▁▁▂▁▂▁▁▂▁▁▁
train/loss_rpn_cls,█▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s0.acc,▁▄▄▄▃▄▄▄▅▄▅▅▅▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███▇███
train/s0.loss_bbox,▅██▆▇▆▅▅▅▅▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▃▂▂▂▁▂▂▁▁▁▁▂▁▁▁
train/s0.loss_cls,█▅▅▄▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train/s1.acc,▄▅▂▃▁▁▂▂▃▂▃▃▃▃▃▄▃▄▄▃▄▄▄▅▅▅▅▇▆▇▇▇▇▇█▇▇███
train/s1.loss_bbox,▁▅█▇█▇▇▇▆▆▆▅▆▆▅▅▆▅▅▅▄▅▅▄▄▅▄▃▄▃▃▃▃▃▃▃▃▃▂▃
